In [151]:
import os
import pandas as pd
from nltk.corpus import stopwords
from pymorphy3 import MorphAnalyzer
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

from typing import List

In [152]:
patterns = "[«»A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()

In [153]:
def create_dataframe_from_csv(csv_path: str) -> pd.DataFrame:
    df_csv = pd.read_csv(csv_path)
    txt = []
    for absolute_path, rating in zip(df_csv['absolute_path'], df_csv['rating']):
        # Чтение текста из файла
        with open(absolute_path, 'r', encoding='utf-8') as file:
            text = file.read()
            # Добавление текста и рейтинга в список
            txt.append((text, rating)) 
    return pd.DataFrame(txt, columns=['review', 'rating'])

In [154]:
df = create_dataframe_from_csv(os.path.join("/Users", "wardonne", "Desktop", "Lab_python", "PP_2_Lab", "1.csv"))
df['rating'] = df['rating'].replace(1,0).replace(2,1).replace(3,2).replace(4,3).replace(5,4)
print(df)

                                                 review  rating
0     За стенкой. История Отиса Ревиаля\nПлюсы:-Прия...       0
1     Её тёмные крылья\nПрочитала, дочитала, вымучил...       0
2     Тринадцать\nДичайше скучная история. Как миним...       0
3     Моя дорогая Оли...\nИ вот опять: с каждого угл...       0
4     Почти идеальный брак\nИтак, рецензии пишу редк...       0
...                                                 ...     ...
4995  До свидания, Рим!\nЛёгкая книга, показывающая ...       4
4996  Оппенгеймер. Триумф и трагедия Американского П...       4
4997  Теория бесконечных обезьян\nПокупая первую для...       4
4998  Чёрные кувшинки\nИстория убийств в уютной фран...       4
4999  Бедовый месяц\nДо сих пор не понимаю, что это ...       4

[5000 rows x 2 columns]


In [155]:
def preprocess_and_lemmatize(review: str) -> str:
    review = re.sub(patterns, ' ', review)
    tokens = review.lower().split()
    preprocessed_text = []
    for token in tokens:
        lemma = morph.parse(token)[0].normal_form
        if lemma not in stopwords_ru:
            preprocessed_text.append(lemma)
    return ' '.join(preprocessed_text)

In [156]:
df['review'] = df['review'].apply(preprocess_and_lemmatize)
print(df.head())

                                              review  rating
0  стенка история отис ревиаля плюс приятный слог...       0
1  её тёмный крыло прочитать дочитать вымучить из...       0
2  тринадцать дикий скучный история минимум треть...       0
3  дорогой оля каждый угол кричать книга восхваля...       0
4  идеальный брак итак рецензия писать редко прав...       0


In [157]:
vectorizer = CountVectorizer(max_features=15000)

# Преобразование данных в матрицу
sparse_matrix = vectorizer.fit_transform(df['review']).toarray()

# Разделение данных
x_train, x_test_valid, y_train, y_test_valid = train_test_split(
    sparse_matrix, np.array(df['rating']), test_size=0.2
)
x_test, x_valid, y_test, y_valid = train_test_split(
    x_test_valid, y_test_valid, test_size=0.5
)

In [158]:
class_counts = dict(zip(*np.unique(y_train, return_counts=True)))
print(class_counts)

{0: 810, 1: 816, 2: 787, 3: 793, 4: 794}
